In [21]:
import os
import sys

In [5]:
src_path = os.path.abspath("").replace("/src/langchain_basics", "")
if src_path not in sys.path:
    sys.path.append(src_path)

In [6]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain, FewShotPromptTemplate
from src.configuration import configuration as cfg

(The following code is based off of https://github.com/samwit/langchain-tutorials/blob/main/)

# Basic LLM setup and usage

In [9]:
llm = LlamaCpp(model_path=os.path.join(cfg.PATHS.TEXTGENERATION_MODEL_PATH,
                            "eachadea_ggml-vicuna-7b-1.1/ggml-vicuna-7b-1.1-q4_0.bin"),
               temperature=0.9,
               max_tokens = 256)

llama.cpp: loading model from /mnt/Workspaces/Workspaces/python/generative-ai-testbench/machine_learning_models/MODELS/eachadea_ggml-vicuna-7b-1.1/ggml-vicuna-7b-1.1-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 4 (mostly Q4_1, some F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 1026.00 MB per state)
llama_init_from_file: kv self size  =  256.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | N

In [10]:
text = "Why did the duck cross the road?"

print(llm(text))


To get to the other side and cool off!
Why was the scarecrow so sad?
Because he had no brain.


# Promt Templates

In [28]:
restaurant_template = """
I want you to act as a naming consultant for new restaurants.
Return a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.
What are some good names for a restaurant that is {restaurant_desription}?
"""

prompt_template = PromptTemplate(
    input_variables=["restaurant_desription"],
    template=restaurant_template
)

In [29]:
description = "a Greek place that serves fresh lamb souvlakis and other Greek food "
description_02 = "a burger place that is themed with baseball memorabilia"
description_03 = "a cafe that has live hard rock music and memorabilia"

## to see what the prompt will be like
prompt_template.format(restaurant_desription=description)

'\nI want you to act as a naming consultant for new restaurants.\nReturn a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.\nWhat are some good names for a restaurant that is a Greek place that serves fresh lamb souvlakis and other Greek food ?\n'

In [30]:
## querying the model with the prompt template
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain only specifying the input variable.
print(chain.run(description_03))

Llama.generate: prefix-match hit


Rock N' Roll Cafe and Hard Rock Café come to mind.


# Using Few Shot Learning

In [23]:
# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [24]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [25]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)

In [26]:
# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:


In [31]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)

# Run the chain only specifying the input variable.
print(chain.run("Big"))

Llama.generate: prefix-match hit


 Small


Word: beautiful
Antonym: ugly

Word: right
Antonym: wrong

Word: hot
Antonym: cold

Word: left
Antonym: right

Word: up
Antonym: down

Word: long
Antonym: short

Word: in
Antonym: out

Word: outside
Antonym: inside

Word: heavy
Antonym: light

Word: fast
Antonym: slow

Word: good
Antonym: bad

Word: upwards
Antonym: downwards

Word: forward
Antonym: backward

Word: outward
Antonym: inward

Word: central
Antonym: peripheral

Word: internal
Antonym: external

Word: digital
Antonym: analog
